In [4]:
import pandas as pd

df = pd.read_csv('../data/processed/mbajk/DVORANA TABOR.csv')
df.shape   

(148, 11)

In [ ]:
# temperature', 'relative_humidity', 'dew_point',
# 'apparent_temperature', 'precipitation_probability', 'rain',
#  'surface_pressure'